# Molecule-field interaction potential

The molecule-field interaction Hamiltonian can be constructed as a sum of products of various interaction tensors (multipoles) with external electric and/or magnetic field. The interaction tensors can be computed using the basis of rotational solutions, see "Rotational dynamics Quickstart" and  "Rotational solutions and matrix elements", or using the ro-vibrational solutions via external programs, such as [TROVE](https://github.com/Trovemaster/TROVE).

We start from computing the rotational solutions and matrix elements for water molecule

In [1]:
from richmol.rot import Molecule, solve, LabTensor
import numpy as np

water = Molecule()

water.XYZ = ("bohr",
             "O",  0.00000000,   0.00000000,   0.12395915,
             "H",  0.00000000,  -1.43102686,  -0.98366080,
             "H",  0.00000000,   1.43102686,  -0.98366080)

# molecular-frame dipole moment (au)
water.dip = [0, 0, -0.7288]

# molecular-frame polarizability tensor (au)
water.pol = [[9.1369, 0, 0], [0, 9.8701, 0], [0, 0, 9.4486]]

Jmax = 10
water.sym = "C2v"

sol = solve(water, Jmax=Jmax)

# laboratory-frame dipole moment operator
dip = LabTensor(water.dip, sol)

# laboratory-frame polarizability tensor
pol = LabTensor(water.pol, sol)

# field-free Hamiltonian
h0 = LabTensor(water, sol)